In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import time

from PIL import Image

import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.utils.data as data
import torch.nn.functional as F

import torchvision
from torchvision import transforms
from torchvision import models
from torch.optim import lr_scheduler

In [2]:
class mydata(data.Dataset):
    def __init__(self, image_path, label_path, filename, labelname):
       
        self.root = image_path
        df = pd.read_csv(label_path) 
        
        self.image_names  = df[filename].values
        self.image_angles = df[labelname].values
               
                
    def __len__(self):
        return len(self.image_names)
    
    
    def __getitem__(self, index):
        image_name = os.path.join(self.root, self.image_names[index])
        image = Image.open(image_name)
        degree = self.image_angles[index]
        
        random_degree = np.random.randint(-15,15)
        image = transforms.functional.rotate(image, random_degree)
        degree += random_degree
        
        pro = transforms.Compose([transforms.ToTensor(),
                                         transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.225, 0.225, 0.225])])
        image_norm = pro(image)
                
        return image_norm, degree

In [3]:
batch_size= 16
train_set = mydata('Homework_4_files/train',
                   'Homework_4_files/train_labels.csv',
                   'filename',
                   'true_rotation')

trainloader = torch.utils.data.DataLoader(dataset = train_set , batch_size= batch_size , shuffle = True)

valid_set = mydata('Homework_4_files/valid',
                   'Homework_4_files/valid_labels.csv',
                   'filename',
                   'true_rotation')
validloader = torch.utils.data.DataLoader(dataset = valid_set , batch_size= batch_size , shuffle = True)



In [5]:
run_id ='five'
os.mkdir(run_id)
criterion=nn.MSELoss(reduction='sum')
loss_name='MSE'
weight_init='Random'
bias_init='Random'
learning_rate=0.01
num_epoch=100
optimizer_name='Adam'

# record all hyperparameters that might be useful to reference later
with open(run_id + '/hyperparams.csv', 'w') as wfil:
    wfil.write("weight initialization," + weight_init + '\n')
    wfil.write("bias initialization," + bias_init + '\n')
    wfil.write("loss function," + loss_name + '\n')
    wfil.write("learning rate," + str(learning_rate) + '\n')
    wfil.write("batch size," + str(batch_size) + '\n')
    wfil.write("number epochs," + str(num_epoch) + '\n')
    wfil.write("optimizer," + optimizer_name + '\n')

# use resnet-18
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, 1)

lis=[{'params':model.layer3.parameters(),'lr':0.0001},
     {'params':model.layer4.parameters(),'lr':0.001},
     {'params':model.fc.parameters(),'lr':0.001}
]
optimizer = optim.Adam(lis)
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

cuda = torch.cuda.is_available()
if cuda:
    # set the network to use cuda
    model = model.cuda()
    print("CUDA IS AVAILABLE!")
else:
    print("CUDA NOT AVAILABLE!")
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# modify the model for regression, the last layer have 1 neuron for regression
# activation is not need

CUDA IS AVAILABLE!


In [6]:
with open(run_id + '/log_file.csv', 'w') as log_fil:
    log_fil.write('epoch,train loss,valid loss\n')
    start = time.time()
    
    for epoch in range(150):

        # track train and validation loss
        epoch_train_loss = 0.0
        epoch_valid_loss = 0.0
        epoch_start = time.time()

    
        for i, (inputs,labels) in enumerate(trainloader):
            # zero out gradients for every batch or they will accumulate
            inputs, labels = torch.autograd.Variable(inputs).cuda(), torch.autograd.Variable(labels).cuda()
            inputs=inputs.float()
            labels=labels.float()

            optimizer.zero_grad()

            # forward step
            outputs = model(inputs)
            outputs=outputs.reshape(outputs.shape[0])
            labels=labels.reshape(labels.shape[0])

            # compute loss
            loss = criterion(outputs, labels)

            # backwards step
            loss.backward()
            # update weights and biases
            optimizer.step()

            # track training loss
            epoch_train_loss += loss.item()

            # calculate training accuracy
        print('epoch is',epoch)
        loss = epoch_train_loss/len(trainloader.sampler)
        print("Training loss is ",loss)
    
        scheduler.step()




        # track valid loss - the torch.no_grad() unsures gradients will not be updated based on validation set
        with torch.no_grad():
            for i, (inputs,labels) in enumerate(validloader):
            # zero out gradients for every batch or they will accumulate
                inputs, labels = torch.autograd.Variable(inputs).cuda(), torch.autograd.Variable(labels).cuda()
                inputs=inputs.float()
                labels=labels.float()
                outputs = model(inputs)
                outputs=outputs.reshape(outputs.shape[0])
                labels=labels.reshape(labels.shape[0])
            
                valid_loss = criterion(outputs, labels)
                epoch_valid_loss += valid_loss.item()
            
            validloss = epoch_valid_loss/len(validloader.sampler)
            print("Validation loss is ",validloss)

        # save best weights
        if best_valid_loss == "unset" or validloss < best_valid_loss:
            best_valid_loss = validloss
            torch.save(model, run_id + "/best_weights.pth")
        
        # save most recent weights
        torch.save(model, run_id + "/last_weights.pth")
        
        log_fil.write(str(epoch) + ',' + str(loss) + ',' + str(validloss) + '\n')
        epoch_end = time.time()
        print("one epoch takes:", epoch_end - epoch_start)
    
    end = time.time()
    print(f"The total time is {(end-start)/60} min") 

epoch is 0

Training loss is  416.55162954486906
Validation loss is  334.5440760117311
one epoch takes: 58.93711614608765
epoch is 1

Training loss is  270.3097634798785
Validation loss is  301.25728188455105
one epoch takes: 41.86299753189087
epoch is 2

Training loss is  199.53422446846963
Validation loss is  159.61702810127574
one epoch takes: 42.11137557029724
epoch is 3

Training loss is  137.59467604974907
Validation loss is  170.32923393409985
one epoch takes: 42.27041244506836
epoch is 4

Training loss is  97.25005077401796
Validation loss is  98.94901271343518
one epoch takes: 42.25245118141174
epoch is 5

Training loss is  78.40252230256796
Validation loss is  99.66861646192578
one epoch takes: 42.43954133987427
epoch is 6

Training loss is  74.08332165946562
Validation loss is  73.24001718025941
one epoch takes: 42.21931004524231
epoch is 7

Training loss is  85.08556317056218
Validation loss is  60.96363259959393
one epoch takes: 42.368205308914185
epoch is 8

Training loss

epoch is 67

Training loss is  12.095753305504719
Validation loss is  34.58370864376999
one epoch takes: 42.07512927055359
epoch is 68

Training loss is  13.06570172842592
Validation loss is  29.77464400117214
one epoch takes: 41.900787591934204
epoch is 69

Training loss is  12.096397295628364
Validation loss is  24.280890036804173
one epoch takes: 41.87751078605652
epoch is 70

Training loss is  12.464563234100739
Validation loss is  19.896071801821773
one epoch takes: 41.9207329750061
epoch is 71

Training loss is  14.31068656562517
Validation loss is  30.291627309929865
one epoch takes: 41.99042630195618
epoch is 72

Training loss is  14.749280537121619
Validation loss is  26.70636168222588
one epoch takes: 41.924025535583496
epoch is 73

Training loss is  12.673206864198049
Validation loss is  25.62761742951205
one epoch takes: 41.865017890930176
epoch is 74

Training loss is  12.564985313018163
Validation loss is  20.742835998821718
one epoch takes: 41.87329459190369
epoch is 75


Validation loss is  19.488588717694466
one epoch takes: 41.94241786003113
epoch is 134

Training loss is  5.110824024757991
Validation loss is  20.887306512763296
one epoch takes: 42.344189167022705
epoch is 135

Training loss is  4.353100437835479
Validation loss is  19.27050984478914
one epoch takes: 41.973860025405884
epoch is 136

Training loss is  6.329660395924002
Validation loss is  18.003596274970242
one epoch takes: 41.83320236206055
epoch is 137

Training loss is  5.348098498942951
Validation loss is  21.991558764129877
one epoch takes: 41.80537724494934
epoch is 138

Training loss is  5.39362276022633
Validation loss is  19.583652775752572
one epoch takes: 41.77738118171692
epoch is 139

Training loss is  5.386352292491744
Validation loss is  25.116762607430037
one epoch takes: 41.90536665916443
epoch is 140

Training loss is  5.647374367055794
Validation loss is  22.927768056710754
one epoch takes: 41.906414270401
epoch is 141

Training loss is  4.6933382532000545
Validatio

In [28]:
test_set = mydata('Homework_4_files/test',
                   'Homework_4_files/test_labels.csv',
                   'filename',
                   'true_rotation')

testloader = torch.utils.data.DataLoader(dataset = test_set , batch_size= batch_size , shuffle = True)

In [29]:
weight_fill = "result/best_weights.pth"
model = torch.load(weight_fil)
model.eval()

with torch.no_grad():
    loss = 0
    for inputs,labels in testloader:
        inputs, labels = torch.autograd.Variable(inputs).cuda(), torch.autograd.Variable(labels).cuda()
        inputs=inputs.float()
        labels=labels.float()
        
        outputs = model(inputs)
        outputs=outputs.reshape(outputs.shape[0])
        labels=labels.reshape(labels.shape[0])

        test_loss = criterion(outputs, labels)
        loss += test_loss.item()
        
        
    print('Test loss of the model is',loss/len(testloader))

Test loss of the model is 530.5353099775315
